### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\Treeb\Instacart Basket Analysis 11-20-2023'

In [3]:
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [35]:
df_customers.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [36]:
df_customers.tail()

user_id First Name    Surnam  Gender           STATE  Age date_joined  \
206204   168073       Lisa      Case  Female  North Carolina   44    4/1/2020   
206205    49635     Jeremy   Robbins    Male          Hawaii   62    4/1/2020   
206206   135902      Doris  Richmond  Female        Missouri   66    4/1/2020   
206207    81095       Rose   Rollins  Female      California   27    4/1/2020   
206208    80148    Cynthia     Noble  Female        New York   55    4/1/2020   

        n_dependants fam_status  income  
206204             1    married  148828  
206205             3    married  168639  
206206             2    married   53374  
206207             1    married   99799  
206208             1    married   57095

### Data Wrangling

#### Changing Column Names

In [4]:
df_customers.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)
df_customers.rename(columns = {'STATE' : 'State'}, inplace = True)
df_customers.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)
df_customers.rename(columns = {'fam_status' : 'Marital_Status'}, inplace = True)

In [62]:
df_customers.head()

user_id First Name Last Name  Gender       State  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   number_of_dependants Marital_Status  income  
0                     3        married  165665  
1                     0         single   59285  
2                     2        married   99568  
3                     0         single   42049  
4                     1        married   40374

#### Changing Data Type

In [63]:
df_customers.dtypes

user_id                  int64
First Name              object
Last Name               object
Gender                  object
State                   object
Age                      int64
date_joined             object
number_of_dependants     int64
Marital_Status          object
income                   int64
dtype: object

In [5]:
df_customers['user_id'] = df_customers['user_id'].astype('str')

In [65]:
df_customers['user_id'].dtype

dtype('O')

In [66]:
df_customers.dtypes

user_id                 object
First Name              object
Last Name               object
Gender                  object
State                   object
Age                      int64
date_joined             object
number_of_dependants     int64
Marital_Status          object
income                   int64
dtype: object

### Consistency Checks

#### Testing for null values

In [43]:
df_customers.isnull().sum()

user_id                     0
First Name              11259
Last Name                   0
Gender                      0
State                       0
Age                         0
date_joined                 0
number_of_dependants        0
Marital_Status              0
income                      0
dtype: int64

In [44]:
df_customers.shape

(206209, 10)

In [45]:
df_no_first_name = df_customers[df_customers['First Name'].isnull() == True]

In [46]:
df_no_first_name.head()

user_id First Name Last Name  Gender      State  Age date_joined  \
53    76659        NaN   Gilbert    Male   Colorado   26    1/1/2017   
73    13738        NaN     Frost  Female  Louisiana   39    1/1/2017   
82    89996        NaN    Dawson  Female     Oregon   52    1/1/2017   
99    96166        NaN   Oconnor    Male   Oklahoma   51    1/1/2017   
105   29778        NaN    Dawson  Female       Utah   63    1/1/2017   

     number_of_dependants Marital_Status  income  
53                      2        married   41709  
73                      0         single   82518  
82                      3        married  117099  
99                      1        married  155673  
105                     3        married  151819

I think it's okay to keep in the null values for the first names, since we have last names for all of our customers. Taking out over 11000 rows for something that won't affect the overall analysis seems a bit much. 

#### Finding Duplicates

In [47]:
df_customer_dups = df_customers[df_customers.duplicated()]

In [48]:
len(df_customer_dups)

0

There are no duplicates in this dataset. 

#### Checking for mixed-type data

In [49]:
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_customers[weird]) > 0:
        print(col) 

First Name


In [52]:
len(df_no_first_name)

11259

I changed the 'First Name' column to strings, but changed it back when I saw it got rid of all the null values. When there is a null value, it is the data type 'nonetype' instead of 'string'. So if I changed the 'First Name' data type to 'string', I might not be able to find the null values using the same way I did before if I decide I need to get rid of them. So for now, I'll keep it a mixed-data type. 

In [27]:
#df_customers['First Name'] = df_customers['First Name'].astype('str')

In [67]:
df_customers.dtypes

user_id                 object
First Name              object
Last Name               object
Gender                  object
State                   object
Age                      int64
date_joined             object
number_of_dependants     int64
Marital_Status          object
income                   int64
dtype: object

In [6]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_flags.pkl'))

In [69]:
df_ords_prods_merge.dtypes

order_id                          int64
user_id                           int64
order_number                      int64
orders_day_of_week                int64
order_hour_of_day                 int64
days_since_users_last_order     float64
product_id                        int64
add_to_cart_order                 int64
reordered                         int64
_merge                         category
product_name                     object
aisle_id                          int64
department_id                     int64
prices                          float64
price_range_loc                  object
busiest_day                      object
busiest_days                     object
busiest_period_of_day            object
max_order                         int64
loyalty_flag                     object
average_purchase_price          float64
spending_flag                    object
median_days_between_orders      float64
frequency_flag                   object
dtype: object

### Merge

In [7]:
df_ords_prods_merge['user_id'] = df_ords_prods_merge['user_id'].astype('str')

In [71]:
df_ords_prods_merge.dtypes

order_id                          int64
user_id                          object
order_number                      int64
orders_day_of_week                int64
order_hour_of_day                 int64
days_since_users_last_order     float64
product_id                        int64
add_to_cart_order                 int64
reordered                         int64
_merge                         category
product_name                     object
aisle_id                          int64
department_id                     int64
prices                          float64
price_range_loc                  object
busiest_day                      object
busiest_days                     object
busiest_period_of_day            object
max_order                         int64
loyalty_flag                     object
average_purchase_price          float64
spending_flag                    object
median_days_between_orders      float64
frequency_flag                   object
dtype: object

In [8]:
df_ords_prods_custs_merged = df_ords_prods_merge.merge(df_customers, on = 'user_id')


### Export

In [9]:
df_ords_prods_custs_merged.head()

order_id user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329       1             1                   2                  8   
1   2398795       1             2                   3                  7   
2    473747       1             3                   3                 12   
3   2254736       1             4                   4                  7   
4    431534       1             5                   4                 15   

   days_since_users_last_order  product_id  add_to_cart_order  reordered  \
0                          NaN         196                  1          0   
1                         15.0         196                  1          1   
2                         21.0         196                  1          1   
3                         29.0         196                  1          1   
4                         28.0         196                  1          1   

  _merge  ...         frequency_flag  First Name  Last Name  Gender    State  \
0   both  ...  Non-Frequent customer       Linda     Nguyen  Female  Alabama   
1   both  ...  Non-Frequent customer       Linda     Nguyen  Female  Alabama   
2   both  ...  Non-Frequent customer       Linda     Nguyen  Female  Alabama   
3   both  ...  Non-Frequent customer       Linda     Nguyen  Female  Alabama   
4   both  ...  Non-Frequent customer       Linda     Nguyen  Female  Alabama   

  Age date_joined number_of_dependants  Marital_Status income  
0  31   2/17/2019                    3         married  40423  
1  31   2/17/2019                    3         married  40423  
2  31   2/17/2019                    3         married  40423  
3  31   2/17/2019                    3         married  40423  
4  31   2/17/2019                    3         married  40423  

[5 rows x 33 columns]

In [10]:
df_ords_prods_custs_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_merged.pkl'))